In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time


In [6]:

driver = webdriver.Chrome()
driver.implicitly_wait(10)

keyword = '서울 강남구 맛집'
driver.get(f'https://map.naver.com/p/search/{keyword}?c=12.00,0,0,0,dh')

def to_iframe(frame_name):
    driver.switch_to.default_content()
    time.sleep(1)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, frame_name)))
    driver.switch_to.frame(frame_name)
    print(f"Switched to iframe: {frame_name}")

def get_li():
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'ul li div div div span'))
        )
        li = driver.find_elements(By.CSS_SELECTOR, 'ul li div div div span')
        return li
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error in get_li: {e}")
        return []

def get_store_info(class_name, list_name):
    try:
        c = soup.select_one(f'span.{class_name}').text
        globals()[list_name].append(c)
    except:
        globals()[list_name].append(float('nan'))

def click_page(p):
    try:
        xpath = '//*[@id="app-root"]/div/div[2]/div[2]/a'
        page = driver.find_elements(By.XPATH, xpath)
        page[p-2].click()  # 페이지 버튼 클릭
        time.sleep(2)  # 페이지 로딩 시간 대기
    except Exception as e:
        print(f"Error clicking page {p}: {e}")

result = pd.DataFrame()

for p in range(2, 6):
    addr = []
    tel = []
    urls = []
    category = []
    
    to_iframe('searchIframe')
    li = get_li()
    
    if not li:  # 리스트가 비어 있을 경우 처리
        print(f"No elements found on page {p}")
        continue
    
    li_last = li[-1] if li else None

    while True:
        li = get_li()
        if not li:  # 비어있는 리스트에 접근하지 않도록 처리
            break
        
        if li_last == li[-1]:  # 마지막 요소가 동일하면 종료
            break
        
        li_last = li[-1]
        driver.execute_script("arguments[0].scrollIntoView();", li[-1])
        time.sleep(0.5)  # 스크롤 후 대기 시간 추가

    store = [l.text for l in li]

    for button in li:
        # 버튼 클릭 전에 스크롤하고 JavaScript로 클릭
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)  # 스크롤 후 잠시 대기
        driver.execute_script("arguments[0].click();", button)

        to_iframe('entryIframe')
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        get_store_info('lnJFt', 'category')
        get_store_info('LDgIH', 'addr')
        get_store_info('xlx7Q', 'tel')

        try:
            u = soup.select_one('a.place_bluelink.CHmqa')['href']
            urls.append(u)
        except:
            urls.append(float('nan'))

        to_iframe('searchIframe')

    res = pd.DataFrame([store, category, addr, urls, tel])
    result = pd.concat([result, res], axis=1, ignore_index=True)

    if p <= 4:
        click_page(p+1)

result = result.T
result.to_csv('./crolltest.csv', index=False)

Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe
Switched to iframe: entryIframe
Switched to iframe: searchIframe


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF62F719632+30946]
	(No symbol) [0x00007FF62F6CE3C9]
	(No symbol) [0x00007FF62F5C6FDA]
	(No symbol) [0x00007FF62F5CDA57]
	(No symbol) [0x00007FF62F5D0058]
	(No symbol) [0x00007FF62F65BD7B]
	(No symbol) [0x00007FF62F63CA6A]
	(No symbol) [0x00007FF62F65B041]
	(No symbol) [0x00007FF62F63C813]
	(No symbol) [0x00007FF62F60A6E5]
	(No symbol) [0x00007FF62F60B021]
	GetHandleVerifier [0x00007FF62F84F83D+1301229]
	GetHandleVerifier [0x00007FF62F85BDB7+1351783]
	GetHandleVerifier [0x00007FF62F852A03+1313971]
	GetHandleVerifier [0x00007FF62F74DD06+245686]
	(No symbol) [0x00007FF62F6D758F]
	(No symbol) [0x00007FF62F6D3804]
	(No symbol) [0x00007FF62F6D3992]
	(No

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
chain = ActionChains(driver)
driver.implicitly_wait(10)  # 전역 대기 시간 설정
keyword = '서울 강남구 음식점'  # 검색할 키워드

driver.get(f'https://map.naver.com/p/search/{keyword}?c=12.00,0,0,0,dh')

def get_li():
    try:
        # XPATH를 확인하여 올바른 요소를 선택하도록 수정
        li = driver.find_elements(
            By.XPATH, 
            '/html/body'
        )
        if len(li) == 0:
            print("경고: 요소를 찾을 수 없습니다!")
        return li
    except Exception as e:
        print(f"오류 발생: {e}")
        return []

def to_iframe(frame_name):
    try:
        driver.switch_to.default_content()
        time.sleep(.5)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, f'//*[@id="{frame_name}"]')))
        driver.switch_to.frame(frame_name)
    except Exception as e:
        print(f"iframe '{frame_name}'로 전환에 실패했습니다. 오류: {e}")

def get_store_info(class_name, list_name):
    try:
        c = soup.select_one(f'span.{class_name}').text
        globals()[list_name].append(c)
    except:
        globals()[list_name].append(float('nan'))

def click_page(p):
    try:
        xpath = '//*[@id="app-root"]/div/div[2]/div[2]'
        page = driver.find_element(
            By.XPATH, xpath + f'/a[{p}]')
        page.click()
    except Exception as e:
        print(f"{p} 페이지 클릭에 실패했습니다: {e}")

result = pd.DataFrame()

for p in range(2, 6):
    
    addr = []
    tel = []
    urls = []
    category = []
    
    to_iframe('searchIframe')

    li = get_li()  # 초기 li 값: 요소 리스트 가져오기
    li_last = ''  # 임의의 값

    while len(li) > 0 and li_last != li[-1]:
        li_last = li[-1]
        chain.scroll_to_element(li[-1]).perform()  # 스크롤 내리기
        li = get_li()  # li 업데이트
    
    store = [l.text for l in li]
    
    for button in li:
        button.click()

        to_iframe('entryIframe')
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        get_store_info('lnJFt', 'category')
        get_store_info('LDgIH', 'addr')
        get_store_info('xlx7Q', 'tel')

        try:
            u = soup.select_one('a.place_bluelink.CHmqa')['href']
            urls.append(u)
        except:
            urls.append(float('nan'))

        to_iframe('searchIframe')
        
    res = pd.DataFrame([store, category, addr, urls, tel])
    result = pd.concat([result, res], axis=1, ignore_index=True)
    
    if p <= 4:
        click_page(p + 1)
    else:
        pass

result = result.T
result.to_csv('./crolltest.csv')
